# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# confirm that our new database was created
print(mongo.list_database_names())  # This should show 'uk_food'


['admin', 'config', 'local', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
collection_names = db.list_collection_names()
print("Collections in 'uk_food' database:", collection_names)

Collections in 'uk_food' database: ['establishments']


In [9]:
# review the collections in our new database
collection_names = db.list_collection_names()
print("Collections in 'uk_food' database:", collection_names)

Collections in 'uk_food' database: ['establishments']


In [12]:
# review a document in the establishments collection
establishments = mongo['establishments']
document = establishments.find_one()
pprint(document)

TypeError: 'Collection' object is not callable. If you meant to call the 'find_one' method on a 'Database' object it is failing because no such method exists.

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "146A Plumstead Rd",
    "AddressLine2": "London",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [14]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

TypeError: 'Collection' object is not callable. If you meant to call the 'insert_one' method on a 'Database' object it is failing because no such method exists.

In [ ]:
# Check that the new restaurant was inserted
# Query to find the new restaurant by its name
restaurant_query = {"BusinessName": "Penang Flavours"}

# Retrieve and print the document for 'Penang Flavours'
penang_flavours = establishments.find_one(restaurant_query)


pprint(penang_flavours)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Projection to return only the BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Retrieve and print the document for 'Restaurant/Cafe/Canteen'
business_type_info = establishments.find_one(business_type_query, projection)

pprint(business_type_info)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = business_type_info["BusinessTypeID"]

# Update the BusinessTypeID for 'Penang Flavours'
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"},  # Query to find the restaurant
    {"$set": {"BusinessTypeID": business_type_id}}  # Update operation
)

In [ ]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

# Retrieve and print the updated document
updated_document = establishments.find_one(query)

pprint(updated_document)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the documents matching the query
dover_count = establishments.count_documents(query)

print(dover_count)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(query)

In [ ]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents(query)

print(dover_count)

In [ ]:
# Check that other documents remain with 'find_one'
document = establishments.find_one()
print(document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
# Update the longitude and latitude fields to double data type
update_result = establishments.update_many(
    {},  # This filter matches all documents
    [
        {
            "$set": {
                "geocode.longitude": {"$toDouble": "$geocode.longitude"},
                "geocode.latitude": {"$toDouble": "$geocode.latitude"}
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
result_int_conversion = establishments.update_many(
    {"RatingValue": {"$in": non_ratings}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

In [ ]:
# Check that the coordinates and rating value are now numbers
sample_documents = establishments.find(
    {
        "$or": [
            {"geocode.longitude": {"$type": "double"}},
            {"geocode.latitude": {"$type": "double"}},
            {"RatingValue": {"$type": "int"}},
            {"RatingValue": {"$type": "null"}}
        ]
    },
    {"geocode": 1, "RatingValue": 1}
).limit(10)  # Limit to 10 documents for review

for doc in sample_documents:
    pprint(doc)